In [3]:
import pandas as pd
import pandasql as psql
import time
import math
df_soup0=pd.read_csv("100_Match_Stats_Souporsecretv4(spell).csv")
pd.options.display.max_columns = None
prep_q1 = "SELECT * from df_soup0 where queue not in (\"5v5 ARAM games\", \"URF games\")"
df_soup = psql.sqldf(prep_q1, locals())
df_soup.head(3)

,minute,participantId,totalGold,level,xp,minionsKilled,jungleMinionsKilled,gameId,queue,teamId,win,champion_name,role,lane,summonerName,pos_x,pos_y,spell0,spell1,item0,item1,item2,item3,item4,item5
0,0,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,14180,14271,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer
1,1,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,6767,11479,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer
2,2,7,584,1,181,3,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,2170,12608,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer


In [4]:
#top_anchor =[0,14600]
#mid_anchor =[7300,7300]
#bot_anchor =[14600,0]
def lane_by_position(df,mins=[4,5,6,7,8],top_anchor = [0,14600], mid_anchor = [7300,7300], bot_anchor = [14600,0]):

    df_mins=df[df['minute'].isin(mins)].copy()
    df_mins['dist_top']=[math.dist([i,j],top_anchor) for i,j in zip(df_mins['pos_x'],df_mins['pos_y'])]
    df_mins['dist_mid']=[math.dist([i,j],mid_anchor) for i,j in zip(df_mins['pos_x'],df_mins['pos_y'])]
    df_mins['dist_bot']=[math.dist([i,j],bot_anchor) for i,j in zip(df_mins['pos_x'],df_mins['pos_y'])]
    df_mins['min_dist']=df_mins[['dist_top','dist_mid','dist_bot']].min(axis=1)

    lane_assignment=[]
    #iterate through each row
    for i in range(len(df_mins)):

        # for each row, check which distance is = to the minimum distances
        if df_mins.iloc[i].dist_bot==df_mins.iloc[i].min_dist:
            lane_assignment.append("BOTTOM")
            
        elif df_mins.iloc[i].dist_mid==df_mins.iloc[i].min_dist:
            lane_assignment.append("MIDDLE")
            
        elif df_mins.iloc[i].dist_top==df_mins.iloc[i].min_dist:
            lane_assignment.append("TOP")

    #lane_assignment
    df_mins['pos_lane']=lane_assignment
    df_mins.head(15)
    
    #make df smaller for group by
    df_mins2=df_mins[['champion_name','gameId','pos_lane']].copy()
    #take the mode of where the champion's calculated lane should be'
    position_flag_df=df_mins2.groupby(['champion_name','gameId']).agg(lambda x:x.value_counts().index[0])
    
    df_merge=df.merge(position_flag_df,on=['champion_name','gameId'])
    #want to return OG dataframe, but with another column of the new mapping
    return df_merge


beg=time.perf_counter()
w = lane_by_position(df_soup)
end=time.perf_counter()
runtime= end-beg
print(int(runtime/60), " minutes and ", runtime%60, "seconds. ")
w.head(5)

0  minutes and  3.2700124000000415 seconds. 


,minute,participantId,totalGold,level,xp,minionsKilled,jungleMinionsKilled,gameId,queue,teamId,win,champion_name,role,lane,summonerName,pos_x,pos_y,spell0,spell1,item0,item1,item2,item3,item4,item5,pos_lane
0,0,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,14180,14271,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,TOP
1,1,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,6767,11479,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,TOP
2,2,7,584,1,181,3,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,2170,12608,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,TOP
3,3,7,839,3,661,11,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,1996,12305,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,TOP
4,4,7,1106,3,1054,17,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,7570,13600,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,TOP


In [19]:
# Classify the Jungler by Smite!
def jungler_by_smite(df):

        #prep_q3 = "SELECT distinct champion_name, gameId, spell0, spell1 FROM df_soup where queue not in (\"5v5 ARAM games\", \"URF games\")"
        #prep3 = psql.sqldf(prep_q3, locals())
        #prep3
        # "Smite"

        df['all_spells'] = df.spell0.map(str) + df.spell1.map(str)

        jungle_flag=[]
        jungle=False
        for j in range(len(df)):
            
            if "Smite" in df.iloc[j]['all_spells']:
                jungle=True
            jungle_flag.append(jungle)
            jungle=False
        df['jungle_flag']=jungle_flag
        df.drop(['all_spells'], axis=1,inplace=True)

        #list of gameIds to loop thru
        gameIdList=list(df['gameId'].drop_duplicates())
        #check for games with more than 2 jungles
        for id in gameIdList:
           
            a=df[(df['gameId'] == id) & (df['minute'] == 4) &  (df['jungle_flag']==True)]

            for i in range(len(a)-2): #doesn't run if 2 junglers

                #sort junglers by jungle cs. chop off jungler   with least cs, until there are only 2 junglers
                a=df[(df['gameId'] == id) & (df['minute'] == 4) &  (df['jungle_flag']==True)]
                b=a[['jungleMinionsKilled','champion_name']].sort_values(['jungleMinionsKilled'],ascending=True)

                #replace jungle flag for all gameId, champion names for top index
                df.loc[(df['gameId'] == id) & (df['champion_name']== df.at[b.index[0],'champion_name']),'jungle_flag']=False

        return df

beg=time.perf_counter()
w2=jungler_by_smite(w)
end=time.perf_counter()
runtime=end-beg
print(int(runtime/60), " minutes and ", runtime%60, "seconds. ")
w2.head(5)

0  minutes and  4.345900700000129 seconds. 


,minute,participantId,totalGold,level,xp,minionsKilled,jungleMinionsKilled,gameId,queue,teamId,win,champion_name,role,lane,summonerName,pos_x,pos_y,spell0,spell1,item0,item1,item2,item3,item4,item5,pos_lane,jungle_flag
0,0,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,14180,14271,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,TOP,False
1,1,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,6767,11479,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,TOP,False
2,2,7,584,1,181,3,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,2170,12608,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,TOP,False
3,3,7,839,3,661,11,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,1996,12305,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,TOP,False
4,4,7,1106,3,1054,17,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,7570,13600,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,TOP,False


In [34]:
# Classify the Support by item!
def support_by_item(df):
      #prep_q2 = "SELECT distinct champion_name, gameId, item0, item1, item2, item3, item4, item5 FROM df_soup where queue not in (\"5v5 ARAM games\", \"URF games\")"
      #prep2 = psql.sqldf(prep_q2, locals())

      supp_items=[ 'Frostfang','Shard of True Ice',  'Steel Shoulderguards',  'Runesteel Spaulders',  'Pauldrons of Whiterock',  'Relic Shield',
        "Targon's Buckler", 'Bulwark of the Mountain',  'Spectral Sickle', 'Harrowing Crescent', 'Black Mist Scythe']
      #should only need to check 1 row per champion / game, so let's filter on only 1 minute
      df2=df[df['minute'].isin([1])].copy()
      df2['all_items'] = df2.item0.map(str) + df2.item1.map(str) + df2.item2.map(str) + df2.item3.map(str) + df2.item4.map(str) + df2.item5.map(str) 

      support_flag=[]
      support=False
      for j in range(len(df2)):
          for si in supp_items:
              if si in df2.iloc[j]['all_items']:
                  support=True
          support_flag.append(support)
          support=False
      df2['support_flag']=support_flag
      
      #merge with original df, so that we just return the OG + the new flag
      df3=df.merge(df2[['champion_name','gameId','support_flag']],on=['champion_name','gameId'])

      #loop thru all games
      gameIdList=list(df3['gameId'].drop_duplicates())
      for id in gameIdList:

        #for the game, grab all bot laners to check for cases where there isn't 2 supports & 2 adcs
        adc=df3[(df3['gameId'] == id) & (df3['minute'] == 8) &  (df3['support_flag']==False) & (df3['pos_lane']=="BOTTOM") & (df3['jungle_flag']==False)]
        supp=df3[(df3['gameId'] == id) & (df3['minute'] == 8) &  (df3['support_flag']==True) & (df3['jungle_flag']==False)]
        #print(len(adc), " ", len(supp))
        if (len(adc) + len(supp)==4): # Only do if there are 4 bot laners. I don't wanna mess with another weird case
          if len(adc)>2: #doesn't run if only 2 (or fewer) ADCs
            for i in range(len(adc)-2): 
                print("ADC Reclassified to support! Game ID: ", id)
                #sort bot laners by minions killed. chop off ADC with least cs, until there are only 2 ADCs

                adc=df3[(df3['gameId'] == id) & (df['minute'] == 8) &  (df3['support_flag']==False) & (df3['pos_lane']=="BOTTOM") & (df3['jungle_flag']==False)]
                b=adc[['minionsKilled','champion_name']].sort_values(['minionsKilled'],ascending=True)
                #print(adc)
                #replace suppport flag for all gameId, champion names for top index. 
                # e.g. someone was mislabeled as an adc when rlly they were the support due to selling supp item
                df3.loc[(df3['gameId'] == id) & (df3['champion_name']== df3.at[b.index[0],'champion_name']),'support_flag']=True
         
          elif len(supp)>2: #doesn't run if only 2 (or fewer) Supports
            for i in range(len(supp)-2): 
                print("Support Reclassified to ADC! Game ID: ", id)
                #sort bot laners by minions killed. chop off support  with most cs, until there are only 2 supports
                supp=df3[(df3['gameId'] == id) & (df['minute'] == 8) &  (df3['support_flag']==True) & (df3['jungle_flag']==False)]
                b=supp[['minionsKilled','champion_name']].sort_values(['minionsKilled'],ascending=False)

                #replace suppport flag for all gameId, champion names for top index. 
                #(someone was mislabeled as an support when rlly they were the adc due to both having a support item) (????? this probably won't ever happen)
                df3.loc[(df3['gameId'] == id) & (df3['champion_name']== df3.at[b.index[0],'champion_name']),'support_flag']=False
      
      return df3
      
beg=time.perf_counter()
w3=support_by_item(w2)
end=time.perf_counter()
runtime=end-beg
print(int(runtime/60), " minutes and ", runtime%60, "seconds. ")
#w3.head(5)

ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
0  minutes and  2.0928851999997278 seconds. 


In [74]:
def new_lane_classification(df,mins=[3,4,5,6,7,8,9,10],top_anchor = [1850,14150], mid_anchor = [8000,8000], bot_anchor = [14150,1850],cleaning=False):
    df_pos=lane_by_position(df, mins, top_anchor, mid_anchor, bot_anchor)

    df_jungle=jungler_by_smite(df_pos)
    df_supp=support_by_item(df_jungle)

    final_role_list=[]
    #change from needing to columns (role & lane) to just one column (TOP, MID , JUNGLE, ADC, SUPPORT)
    for j in range(len(df_supp)):
        #lane=df_jungle.iloc[j]['pos_lane']
        #role=df_jungle.iloc[j]['role']
        final_role=df_supp.iloc[j]['pos_lane']
        if df_supp.iloc[j]['support_flag'] == True:
            final_role = "SUPPORT"
        if df_supp.iloc[j]['jungle_flag'] == True:
            final_role="JUNGLE"
        if (final_role == "BOTTOM"):
            final_role = "BOT_CARRY"
        
        final_role_list.append(final_role)

    df_supp['final_role']=final_role_list

    #flag games with incorrect role classifcation (checks for 2 members in each role per game)
    if cleaning:
        role_df=df_supp[['gameId','minute','final_role']]
        role_df_2=role_df[role_df['minute']==1][['gameId','final_role']]
        role_df_2['cnt']=[1 for i in range(len(role_df_2))]
        role_df_3=role_df_2.groupby(['gameId','final_role']).sum()
        #should be 2 roles for each position each game. "diff" column tells us if there how far off we are from 2
        role_df_3['diff']=abs(role_df_3['cnt']-2)
        #reset index so we can grab the gameId column after the groupby
        role_df_3.reset_index(inplace=True)
        #Get a list of just gameIds with a diff
        diff_df = role_df_3[['gameId','diff']].copy().drop_duplicates()
        diff_df_2=diff_df[diff_df['diff']>0].copy()
        #Assign value of True to games wiht a classification error
        diff_df_2['role_error_flag']=[True for i in range(len(diff_df_2))]
        diff_df_3=diff_df_2[['gameId','role_error_flag']].copy().drop_duplicates()
        
        df_final=df_supp.merge(diff_df_3,how='left',on=['gameId'])
        #this fills the null values from the role_error_flag merge to be False
        #Also fills in the item columns which has no items in it

        df_final.fillna(value=False, inplace=True)
        return df_final
    else:
        return df_supp 

beg=time.perf_counter()
w4=new_lane_classification(df_soup,mins=[3,4,5,6,7,8,9,10],top_anchor = [1850,14150], mid_anchor = [8000,8000], bot_anchor = [14150,1850], cleaning=True)
end=time.perf_counter()
runtime=end-beg
print(int(runtime/60), " minutes and ", runtime%60, "seconds. ")
w4.head(5)

ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
0  minutes and  20.745003999996698 seconds. 


,minute,participantId,totalGold,level,xp,minionsKilled,jungleMinionsKilled,gameId,queue,teamId,win,champion_name,role,lane,summonerName,pos_x,pos_y,spell0,spell1,item0,item1,item2,item3,item4,item5,jungle_flag,pos_lane,support_flag,final_role,role_error_flag
0,0,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,14180,14271,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False,TOP,False,TOP,False
1,1,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,6767,11479,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False,TOP,False,TOP,False
2,2,7,584,1,181,3,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,2170,12608,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False,TOP,False,TOP,False
3,3,7,839,3,661,11,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,1996,12305,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False,TOP,False,TOP,False
4,4,7,1106,3,1054,17,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,7570,13600,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False,TOP,False,TOP,False


In [77]:
sv=w4[['gameId','teamId','champion_name','summonerName','role','lane','final_role']].drop_duplicates()
print(score_error(w4))
w4.to_csv("New_Lane_data_4_14.csv",index=False)
#sv.head(11)
#a=w4[(w4['gameId'] == 3776024272) & (w4['minute'] == 3) ]
a=w4[(w4['role_error_flag'] == True) & (w4['minute'] == 3) ]
a

4


,minute,participantId,totalGold,level,xp,minionsKilled,jungleMinionsKilled,gameId,queue,teamId,win,champion_name,role,lane,summonerName,pos_x,pos_y,spell0,spell1,item0,item1,item2,item3,item4,item5,jungle_flag,pos_lane,support_flag,final_role,role_error_flag
23103,3,9,797,2,375,9,0,3776024272,5v5 Ranked Flex games,Red,0,Azir,DUO,NONE,DJ Brownie,6954,7065,Flash,Teleport,False,Corrupting Potion,False,False,False,False,False,MIDDLE,False,MIDDLE,True
23108,3,1,755,2,602,6,0,3776024272,5v5 Ranked Flex games,Blue,1,Chogath,DUO,NONE,Teroboal,1197,11568,Teleport,Flash,Doran's Shield,False,False,False,False,False,False,TOP,False,TOP,True
23113,3,10,948,3,904,15,0,3776024272,5v5 Ranked Flex games,Red,0,Irelia,DUO,NONE,hi lm teddy,2506,11976,Flash,Teleport,Doran's Blade,False,False,False,False,False,False,TOP,False,TOP,True
23118,3,5,962,3,875,16,0,3776024272,5v5 Ranked Flex games,Blue,1,Kaisa,DUO,NONE,AerialBlast,13404,3466,Heal,Flash,Doran's Blade,Health Potion,False,False,False,False,False,BOTTOM,False,BOT_CARRY,True
23123,3,3,843,2,490,0,8,3776024272,5v5 Ranked Flex games,Blue,1,Kindred,DUO,NONE,radaemon1,2488,11960,Smite,Flash,Emberknife,Refillable Potion,False,False,False,False,True,TOP,False,JUNGLE,True
23128,3,4,759,2,395,3,0,3776024272,5v5 Ranked Flex games,Blue,1,Leona,DUO,NONE,SouporSecret,6332,5953,Ignite,Flash,Relic Shield,False,False,False,False,False,False,MIDDLE,True,SUPPORT,True
23133,3,2,843,3,666,9,0,3776024272,5v5 Ranked Flex games,Blue,1,Lux,DUO,NONE,koreazian,5674,6486,Barrier,Flash,Doran's Ring,False,False,False,False,False,False,MIDDLE,False,MIDDLE,True
23138,3,6,752,2,375,4,0,3776024272,5v5 Ranked Flex games,Red,0,Maokai,DUO,NONE,BallDipper2000,7193,6716,Flash,Ignite,Doran's Shield,False,False,False,False,False,False,MIDDLE,False,MIDDLE,True
23143,3,8,937,2,375,0,0,3776024272,5v5 Ranked Flex games,Red,0,Senna,DUO,NONE,Yatogamí,6935,6292,Heal,Flash,Spectral Sickle,False,False,False,False,False,False,MIDDLE,True,SUPPORT,True
23148,3,7,643,1,0,0,0,3776024272,5v5 Ranked Flex games,Red,0,Volibear,DUO,NONE,Crab Bo,14340,14390,Teleport,Flash,False,False,False,False,False,False,False,MIDDLE,False,MIDDLE,True


In [37]:
#See how close we are to 2 ppl in each position, score for each wrong member
#@Param df = dataframe after being run through the classifcation
def score_error(df):

    role_df=df[['gameId','minute','final_role']]

    num_games=len(role_df[['gameId']].copy().drop_duplicates())

    role_df_2=role_df[role_df['minute']==1][['gameId','final_role']]

    #arbitrary counter column with 1s in it, used to aggregate # of positions in each game
    role_df_2['cnt']=[1 for i in range(len(role_df_2))]
    role_df_3=role_df_2.groupby(['gameId','final_role']).sum()
    #should be 2 roles for each position each game. "diff" column tells us if there how far off we are from 2
    role_df_3['diff']=abs(role_df_3['cnt']-2)
    #should be num_games * 5 rows in this table. a missing row means a "diff" of 2
    score=role_df_3['diff'].sum() + ((num_games*5) - len(role_df_3['diff']))*2
    return score
print(score_error(w4))#Yes! matches my excel file!

4


In [38]:
scores=[]
#original map size estimage
df0=new_lane_classification(df_soup,mins=[4,5,6,7,8],top_anchor = [0,14600], mid_anchor = [7300,7300], bot_anchor = [14600,0])
df1=new_lane_classification(df_soup,mins=[4,5,6,7,8,9,10],top_anchor = [0,14600], mid_anchor = [7300,7300], bot_anchor = [14600,0])
df2=new_lane_classification(df_soup,mins=[4,5,6],top_anchor = [0,14600], mid_anchor = [7300,7300], bot_anchor = [14600,0])
#turns out map might actually be 16,000 by 16,000
df3=new_lane_classification(df_soup,mins=[4,5,6,7,8],top_anchor = [0,16000], mid_anchor = [8000,8000], bot_anchor = [16000,0])
df4=new_lane_classification(df_soup,mins=[4,5,6,7,8,9,10],top_anchor = [0,16000], mid_anchor = [8000,8000], bot_anchor = [16000,0])
df5=new_lane_classification(df_soup,mins=[4,5,6],top_anchor = [0,16000], mid_anchor = [8000,8000], bot_anchor = [16000,0])
#retrying where the middle of the lanes actually are
df6=new_lane_classification(df_soup,mins=[4,5,6,7,8],top_anchor = [1850,14150], mid_anchor = [8000,8000], bot_anchor = [14150,1850])
df7=new_lane_classification(df_soup,mins=[4,5,6,7,8,9,10],top_anchor = [1850,14150], mid_anchor = [8000,8000], bot_anchor = [14150,1850])
df8=new_lane_classification(df_soup,mins=[4,5,6],top_anchor = [1850,14150], mid_anchor = [8000,8000], bot_anchor = [14150,1850])
scores.append(score_error(df0))
scores.append(score_error(df1))
scores.append(score_error(df2))
scores.append(score_error(df3))
scores.append(score_error(df4))
scores.append(score_error(df5))
scores.append(score_error(df6))
scores.append(score_error(df7))
scores.append(score_error(df8))
print(scores)
# scores:  [20, 18, 36, 30, 22, 46, 14, 14, 24]
#takeaway: 7 minutes was best, and final set of best guess anchors are best

ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703


In [39]:
scores2=[]
#add min 3 to everything
#original map size estimage
df_0=new_lane_classification(df_soup,mins=[3,4,5,6,7,8],top_anchor = [0,14600], mid_anchor = [7300,7300], bot_anchor = [14600,0])
df_1=new_lane_classification(df_soup,mins=[3,4,5,6,7,8,9,10],top_anchor = [0,14600], mid_anchor = [7300,7300], bot_anchor = [14600,0])
df_2=new_lane_classification(df_soup,mins=[3,4,5,6],top_anchor = [0,14600], mid_anchor = [7300,7300], bot_anchor = [14600,0])
#turns out map might actually be 16,000 by 16,000
df_3=new_lane_classification(df_soup,mins=[3,4,5,6,7,8],top_anchor = [0,16000], mid_anchor = [8000,8000], bot_anchor = [16000,0])
df_4=new_lane_classification(df_soup,mins=[3,4,5,6,7,8,9,10],top_anchor = [0,16000], mid_anchor = [8000,8000], bot_anchor = [16000,0])
df_5=new_lane_classification(df_soup,mins=[3,4,5,6],top_anchor = [0,16000], mid_anchor = [8000,8000], bot_anchor = [16000,0])
#retrying where the middle of the lanes actually are
df_6=new_lane_classification(df_soup,mins=[3,4,5,6,7,8],top_anchor = [1850,14150], mid_anchor = [8000,8000], bot_anchor = [14150,1850])
df_7=new_lane_classification(df_soup,mins=[3,4,5,6,7,8,9,10],top_anchor = [1850,14150], mid_anchor = [8000,8000], bot_anchor = [14150,1850])
df_8=new_lane_classification(df_soup,mins=[3,4,5,6],top_anchor = [1850,14150], mid_anchor = [8000,8000], bot_anchor = [14150,1850])
scores2.append(score_error(df_0))
scores2.append(score_error(df_1))
scores2.append(score_error(df_2))
scores2.append(score_error(df_3))
scores2.append(score_error(df_4))
scores2.append(score_error(df_5))
scores2.append(score_error(df_6))
scores2.append(score_error(df_7))
scores2.append(score_error(df_8))
print(scores2)
# scores: [10, 10, 12, 10, 10, 12, 10, 10, 12]
#takeaway: Including minute 3 is great, doesn't even matter where the anchors are???

ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
ADC Reclassified to support!!! Game ID:  3830577895
ADC Reclassified to support!!! Game ID:  3804254703
[4, 4, 6, 4, 4, 6, 4, 4, 6]
